# UOV (Unbalanced Oil and Vinegar) Implementation Notebook

## Overview

This notebook explains the key generation, signing, and verification functions for the UOV digital signature scheme. We'll go through each function step-by-step, understanding what it does and how it implements the UOV specification.

In [1]:
from sage.all import *

## 1. The `Upper()` Function

### Explanation

In UOV, we work with quadratic forms represented as matrices. A quadratic form can be written as x^T · M · x where M is a matrix. However, the same quadratic form can be represented by different matrices-specifically, M and its transpose M^T give the same result.

The `Upper()` function converts any square matrix M into an **upper triangular matrix** that defines the same quadratic form. This is done by:
- Taking the sum of symmetric positions: M[i,j] becomes M[i,j] + M[j,i]
- Clearing the lower triangle: M[j,i] becomes 0

This ensures we have a canonical representation with no redundancy.

In [2]:
def Upper(M):
    """
    input:
    M = square matrix
    
    output:
    Upper triangular matrix that defines the same quadratic form as M
    """
    nn = M.ncols()
    limit = 1
    for i in range(nn):
        for j in range(i+limit, nn):
            M[i, j] += M[j, i]  # Combine symmetric entries
            M[j, i] = 0          # Clear lower triangle
    return M

## 2. The `Eval()` Function

### Explanation

To verify a signature or compute values during signing, we need to evaluate quadratic polynomials. Each polynomial in UOV is represented as a matrix M, and we evaluate it at a vector x using the quadratic form: p(x) = x^T · M · x.

The `Eval()` function takes a list of matrices (representing multiple polynomials) and a vector, then returns the vector of all evaluations.

In [3]:
def Eval(F, x):
    """
    input:
    F = list of square matrices of size len(x)
    x = a vector with entries in F_q
    
    output:
    vector with the evaluation at x of the quadratic forms defined by the matrices in F
    """
    return vector([ x*M*x for M in F])

## 3. The `Get_UOV_Keys()` Function

### Explanation

This is the core key generation function. It implements the key generation algorithm from the UOV specification:

**Step-by-step process:**

1. **Set up the field**: Create the finite field F_q with q elements
2. **Generate the Oil space**: 
   - Sample a random (n-m) × m matrix O
   - Create O_bar = [O | I_m]^T (the Oil space matrix with identity appended)
3. **Generate public key matrices**: For each polynomial i ∈ [m]:
   - Sample upper triangular P1 of size (n-m) × (n-m)
   - Sample P2 of size (n-m) × m
   - Compute P3 such that the resulting block matrix has the Oil space in its kernel
   - P3 = Upper(-O^T · P1 · O - O^T · P2)

The public key consists of the block matrices that combine P1, P2, and P3.

In [4]:
def Get_UOV_Keys(n, m, q):
    """
    input:
    n = number of variables
    m = number of polynomials (also = dimension of Oil space)
    q = size of the field F_q
    
    output:
    F_q = field of size q
    P = list with block matrices [P1, P2, P3] components
    Public_key = list with public key matrices (full blocks)
    Public_key_polar = list with polar forms (symmetric versions)
    O = random matrix of order (n-m) × m (trapdoor)
    O_bar = Oil space [O | I_m]^T
    """
    o = m  # Oil dimension equals m
    F_q = GF(q)  # Create finite field
    
    # Generate trapdoor: random (n-m) × m matrix
    O = random_matrix(F_q, n-o, o)
    
    # Create oil space: [O | I_m]^T
    O_bar = block_matrix([[O], [identity_matrix(F_q, o)]])
    
    P = []
    Public_key = []
    Public_key_polar = []
    
    for i in range(0, m):
        # Generate random components
        P1 = Upper(random_matrix(F_q, n-o, n-o))
        P2 = random_matrix(F_q, n-o, o)
        
        # Compute P3 so Oil space vanishes
        P3 = Upper(-O.transpose()*P1*O - O.transpose()*P2)
        
        P.append([P1, P2, P3])
        
        # Construct full block matrix
        matriz = block_matrix([[P1, P2], [zero_matrix(F_q, o, n-o), P3]])
        Public_key.append(matriz)
        
        # Polar form (symmetric): M + M^T
        Public_key_polar.append(matriz + matriz.transpose())
    
    return F_q, P, Public_key, Public_key_polar, O, O_bar

## 4. The `Check_Oil()` Function

### Explanation

This is a verification/debugging function. According to UOV design, the public key polynomials should vanish (evaluate to zero) on the entire Oil space O_bar. This function checks this property by evaluating the public key polynomials at each basis vector of the Oil space.

If the Oil space is correctly hidden in the public key, all evaluations should be zero vectors.

In [ ]:
def Check_Oil(Public_key, O_bar):
    """
    input:
    Public_key = list with public key matrices
    O_bar = Oil space of the UOV map = [O | I_m]^T
    
    output:
    evaluation of the public key polynomials at every basis vector of O_bar
    (should all be zero vectors if construction is correct)
    """
    o = O_bar.ncols()
    
    # Evaluate at each basis vector of Oil space
    return [Eval(Public_key, O_bar.transpose()[i]) for i in range(o)]

## 5. The `Sign()` Function

### Explanation

This implements the UOV signing algorithm using the hash-and-sign paradigm:

**Algorithm:**
1. Precompute S matrices: S_i = (P1_i + P1_i^T) · O + P2_i
2. Loop (rejection sampling):
   - Sample a random vinegar vector v ∈ F_q^{n-m}
   - Build matrix L where each row i is: v^T · S_i
   - Compute y: the evaluation of the central map at [v, 0]
   - Solve the linear system: L · x = t - y
   - If solvable, construct signature: s = [v | 0] + O_bar · x
   - If not solvable, try again with different v

The key insight: by fixing the vinegar variables, we reduce the problem to solving a linear system over the oil variables.

In [ ]:
from typing import Any

def Sign(P, O, O_bar, t_document):
    """
    input:
    P = list with block matrices of the public key
    O = random matrix for the Oil space of size (n-m) × m
    O_bar = Oil space of the UOV map = [O | I_m]^T
    t_document = document in F_q^m to be signed
   
    output:
    signature = signature (in F_q^n) of t_document
    """
    F_q = O.base_ring()
    n = O.nrows() + O.ncols()
    m = O.ncols()
   
    s: Any = None  # Initialize to satisfy linter; will be assigned in loop before return
    
    S = [(PM[0] + PM[0].transpose())*O + PM[1] for PM in P]
    Firma = False
    intentos = 1
   
    while not Firma:
        V = random_matrix(F_q, 1, n-m)
        L = zero_matrix(F_q, m, m)
        y_list = []
       
        for i in range(m):
            L[i] = V * S[i]
            temp = V*P[i][0]*V.transpose()
            y_list.append(temp[0,0])
        
        t_y = vector(t_document) - vector(y_list)
        
        try:
            x = L.solve_right(t_y)
            s = vector([V[0, i] for i in range(n-m)] + [0 for i in range(m)]) + O_bar*x
            break
        except ValueError:
            intentos += 1
   
    print(f"Number of tries = {intentos}")
    return s

## 6. The `Verify()` Function

### Explanation

Verification is straightforward in UOV (as in all hash-and-sign schemes):
1. Evaluate the public key at the signature: y = P(signature)
2. Check if y equals the hash of the document
3. Return True if they match, False otherwise

This works because:
- Only someone with the secret key can efficiently find a preimage of an arbitrary hash
- The public key is a composition P = F ∘ T, where F is hidden

In [7]:
def Verify(Public_key, t_document, signature): 
    """
    input:
    Public_key = list with public key matrices
    t_document = document in F_q^m that was signed
    signature = vector in F_q^n that is a signature of t_document
    
    output:
    Boolean: True if signature is valid, False otherwise
    """
    # Evaluate public key at signature
    y = Eval(Public_key, signature)
    
    # Check if result matches the document hash
    return y == vector(t_document)

## Key Insights

1. **Oil Space Hiding**: The trapdoor is the Oil space O_bar. By construction, all public polynomials vanish on this space, making it difficult to find without knowing O.

2. **Signing Strategy**: Signing works by:
   - Fixing the vinegar variables randomly
   - Reducing the problem to a linear system in oil variables
   - Rejection sampling until the system is invertible

3. **Security vs. Efficiency**: The number of signing attempts typically only takes 1-2 tries because the probability of L being invertible is approximately 1 - 1/q.

4. **Field Arithmetic**: All operations are in the finite field F_q, making the scheme efficient on small fields (like F_256 or F_16).

## Issues & Notes for Improvement

### Issue 1: Missing variable `n` in `Sign()`
**Problem**: The `Sign()` function uses `V = random_matrix(F_q, 1, n-m)` but `n` is never defined as a local variable.

**Fix**: Add at the beginning of the function.

### Issue 2: Unused variable in `Check_Oil()`
**Problem**: The line `v_random = sum(F_q.random_element()*O_bar.transpose()[i] for i in range(o))` computes a random vector but never uses it.

**Fix**: This line can be removed entirely.

### Issue 3: Inconsistent naming in `Sign()`
**Problem**: 
- Variable `V` is actually a 1×(n-m) matrix, not a vector
- Some other variables names

**Fix**: Renaming for clarity.

### Issue 4: No maximum iteration limit in `Sign()`
**Problem**: If the matrix L keeps being singular, the function loops indefinitely (though statistically extremely unlikely with proper parameters).

**Fix**: Add a safety limit.

### Issue 5: No input validation in `Verify()`
**Problem**: The function assumes correct input dimensions without checking. Wrong sized inputs could cause cryptic errors.

**Fix**: Add validation.

### Issue 6: Three redundant public key formats
**Problem**: `Get_UOV_Keys()` returns three versions (P, Public_key, Public_key_polar) which is confusing:
- `Sign()` uses `P` (decomposed form [P1, P2, P3])
- `Verify()` uses `Public_key` (full block form)
- `Public_key_polar` is never used in provided code

**Clarification**: These exist for flexibility:
- **P**: Decomposed form [P1, P2, P3] most efficient for signing operations
- **Public_key**: Full block matrix form-direct representation for verification
- **Public_key_polar**: Symmetric form M + M^T-numerically cleaner for polynomial evaluation

Consider documenting which form should be used when, or simplifying to return only what's needed for your use case.

### Issue 7: `Upper()` function purpose not leveraged
**Problem**: The `Upper()` function creates a canonical upper-triangular form, but this property isn't explicitly used in signing/verification for consistency checking.

**Note**: This is mathematically fine, the math still works correctly even without explicit use. However, it's worth understanding that both `Public_key` and `Public_key_polar` implicitly maintain the upper triangular structure that was enforced during key generation.


**All of this changes are already implemented.**

# Additions:

## 1. Hashing

Let's improve this implementation, starting with hashing the message, as in the documentation.

1. **`Hash_Message()` function** - Uses SHAKE-256 to hash the message + salt into a vector in F_q^m. It converts the hash bytes to field elements modulo q.

2. **Salt handling** - Sign now accepts an optional salt parameter. If not provided, generates a random 16-byte salt. Returns both signature and salt together.

3. **Updated Sign()** - Now takes the actual message string instead of pre-hashed t_document. Computes the hash internally. Also returns the number of attempts for debugging.

4. **Updated Verify()** - Takes the message string and extracts salt from the signature tuple. Recomputes the hash and checks it matches the evaluation of the signature under the public key.

5. **Simplified key generation** - Removed the unused `Public_key_polar` return and the redundant P storage since Sign uses the precomputed S matrices.

6. **Example usage at bottom** - Shows how to sign a message and verify it works, plus a negative test showing wrong messages fail verification.

The cryptographic security now comes from the hash function making it hard to forge signatures for arbitrary messages.

In [8]:
import hashlib

def Upper(M):
    nn = M.ncols()
    limit = 1
    for i in range(nn):
        for j in range(i+limit, nn):
            M[i, j] += M[j, i]
            M[j, i] = 0
    return M

def Eval(F, x):
    return vector([ x*M*x for M in F])

def Hash_Message(message, salt, m, q, debug=False):
    combined = message.encode() if isinstance(message, str) else message
    combined += salt
    
    if debug:
        print(f"[HASH] Message: {message}")
        print(f"[HASH] Salt: {salt.hex()}")
        print(f"[HASH] Combined: {combined.hex()}")
    
    hash_obj = hashlib.shake_256(combined)
    hash_bytes = hash_obj.digest(m * (q.bit_length() // 8 + 1))
    
    if debug:
        print(f"[HASH] Hash bytes: {hash_bytes.hex()}")
    
    F_q = GF(q)
    hash_elements = []
    
    for i in range(m):
        byte_val = int.from_bytes(hash_bytes[i*2:i*2+2], byteorder='big')
        field_elem = F_q(byte_val % q)
        hash_elements.append(field_elem)
        if debug and i < 5:
            print(f"[HASH] Element {i}: byte_val={byte_val}, field_elem={field_elem}")
    
    result = vector(hash_elements)
    if debug:
        print(f"[HASH] Final hash vector: {result[:5]}... (showing first 5)")
    
    return result

def Get_UOV_Keys(n, m, q):
    o = m
    F_q = GF(q)
    O = random_matrix(F_q, n-o, o)
    O_bar = block_matrix([[O], [identity_matrix(F_q, o)]])
    
    P = []
    Public_key = []
    
    for i in range(0, m):
        P1 = Upper(random_matrix(F_q, n-o, n-o))
        P2 = random_matrix(F_q, n-o, o)
        P3 = Upper(-O.transpose()*P1*O - O.transpose()*P2)
        
        P.append([P1, P2, P3])
        
        matriz = block_matrix([[P1, P2], [zero_matrix(F_q, o, n-o), P3]])
        Public_key.append(matriz)
    
    return F_q, P, Public_key, O, O_bar

def Check_Oil(Public_key, O_bar):
    m = len(Public_key)
    o = O_bar.ncols()
    
    return [Eval(Public_key, O_bar.transpose()[i]) for i in range(o)]

def Sign(P, O, O_bar, message, salt=None, debug=False):
    F_q = O.base_ring()
    n = O.nrows() + O.ncols()
    m = O.ncols()

    s: Any = None
    
    if debug:
        print(f"\n[SIGN] Starting sign process")
        print(f"[SIGN] n={n}, m={m}, q={F_q.order()}")
        print(f"[SIGN] O shape: {O.nrows()}x{O.ncols()}")
        print(f"[SIGN] O_bar shape: {O_bar.nrows()}x{O_bar.ncols()}")
    
    if salt is None:
        salt = bytes([randint(0, 255) for _ in range(16)])
    elif isinstance(salt, str):
        salt = salt.encode()
    
    t = Hash_Message(message, salt, m, F_q.order(), debug=debug)
    
    if debug:
        print(f"[SIGN] Target hash t: {t[:5]}... (first 5 elements)")
    
    S = [(PM[0]+PM[0].transpose())*O+PM[1] for PM in P]
    
    if debug:
        print(f"[SIGN] Computed {len(S)} S matrices")
    
    signed = False
    attempts = 1
    max_attempts = 256
    
    while not signed:
        if debug and attempts <= 2:
            print(f"\n[SIGN] Attempt {attempts}:")
        
        v = random_matrix(F_q, 1, n-m)
        L = zero_matrix(F_q, m, m)
        y_list = []
        
        for i in range(m):
            L[i] = v * S[i]
            temp = v*P[i][0]*v.transpose()
            y_list.append(temp[0,0])
        
        if debug and attempts <= 2:
            print(f"[SIGN]   v (first 5): {[v[0,i] for i in range(min(5, n-m))]}")
            print(f"[SIGN]   y_list (first 5): {y_list[:5]}")
            print(f"[SIGN]   L rank: {L.rank()}, L determinant: {L.determinant()}")
        
        t_y = vector(t) - vector(y_list)
        
        if debug and attempts <= 2:
            print(f"[SIGN]   t-y: {t_y[:5]}... (first 5)")
        
        try:
            x = L.solve_right(t_y)
            s = vector([v[0, i] for i in range(n-m)] + [0 for i in range(m)]) + O_bar*x
            
            if debug:
                print(f"[SIGN]   Solved! x (first 5): {x[:5]}")
                print(f"[SIGN]   Signature s (first 5): {s[:5]}")
            
            signed = True
        except ValueError:
            if debug and attempts <= 2:
                print(f"[SIGN]   L not invertible, retrying...")
            attempts += 1
            if attempts > max_attempts:
                raise Exception("Signing failed: couldn't find invertible system after many attempts")
    
    if debug:
        print(f"[SIGN] Signature generated successfully in {attempts} attempt(s)\n")
    
    return (s, salt, attempts)

def Verify(Public_key, message, signature, debug=False):
    s, salt = signature
    
    F_q = Public_key[0].base_ring()
    m = len(Public_key)
    n = Public_key[0].nrows()
    
    if debug:
        print(f"\n[VERIFY] Starting verification")
        print(f"[VERIFY] n={n}, m={m}, q={F_q.order()}")
        print(f"[VERIFY] Message: {message}")
        print(f"[VERIFY] Salt: {salt.hex()}")
    
    if len(s) != n:
        raise ValueError(f"Signature dimension {len(s)} doesn't match expected {n}")
    if F_q.order() != 256 and F_q.order() != 16:
        raise ValueError(f"Unsupported field size")
    
    t = Hash_Message(message, salt, m, F_q.order(), debug=debug)
    
    if debug:
        print(f"[VERIFY] Computed target hash t: {t[:5]}... (first 5 elements)")
    
    y = Eval(Public_key, s)
    
    if debug:
        print(f"[VERIFY] Evaluated P(s): {y[:5]}... (first 5 elements)")
        print(f"[VERIFY] t == P(s): {y == vector(t)}")
        print(f"[VERIFY] Match: {y == vector(t)}\n")
    
    return y == vector(t)


n = 112
m = 44
q = 256

F_q, P, Public_key, O, O_bar = Get_UOV_Keys(n, m, q)

print("="*70)
print("TEST 1: Sign and verify valid message")
print("="*70)

message = "Hello, UOV!"
signature, salt, attempts = Sign(P, O, O_bar, message, debug=True)

print(f"Signature generated in {attempts} attempt(s)")
print(f"Salt: {salt.hex()}")

is_valid = Verify(Public_key, message, (signature, salt), debug=True)
print(f"Signature valid: {is_valid}")

print("\n" + "="*70)
print("TEST 2: Verify fails with wrong message")
print("="*70)

is_valid_wrong = Verify(Public_key, "Hello, World!", (signature, salt), debug=True)
print(f"Wrong message valid: {is_valid_wrong}")

print("\n" + "="*70)
print("TEST 3: Verify fails with tampered salt")
print("="*70)

tampered_salt = bytes([salt[i] ^ 0xFF if i == 0 else salt[i] for i in range(len(salt))])
is_valid_tampered = Verify(Public_key, message, (signature, tampered_salt), debug=True)
print(f"Tampered salt valid: {is_valid_tampered}")

TEST 1: Sign and verify valid message

[SIGN] Starting sign process
[SIGN] n=112, m=44, q=256
[SIGN] O shape: 68x44
[SIGN] O_bar shape: 112x44
[HASH] Message: Hello, UOV!
[HASH] Salt: abfd0861cf5400b6c56227c397ef1de3
[HASH] Combined: 48656c6c6f2c20554f5621abfd0861cf5400b6c56227c397ef1de3
[HASH] Hash bytes: 6f908f7952b02da71bcac53dfabdb2ef14a2a9935ced2458a3082e300510f27d403df5c6f7e66c3e4306423815c11799e08dba1c6b125cd7556c1c27f3b5766192410a462411bf6649a77a24aa0aa42ee0dd4b24404ff952
[HASH] Element 0: byte_val=28560, field_elem=0
[HASH] Element 1: byte_val=36729, field_elem=1
[HASH] Element 2: byte_val=21168, field_elem=0
[HASH] Element 3: byte_val=11687, field_elem=1
[HASH] Element 4: byte_val=7114, field_elem=0
[HASH] Final hash vector: (0, 1, 0, 1, 0)... (showing first 5)
[SIGN] Target hash t: (0, 1, 0, 1, 0)... (first 5 elements)
[SIGN] Computed 44 S matrices

[SIGN] Attempt 1:
[SIGN]   v (first 5): [z8^7 + z8^5 + z8^4 + z8^2, z8 + 1, z8^7 + z8^5 + z8^2 + z8, z8^6 + z8^4 + z8^3 + z8^2,

Everything is working for now.

# 2. Salt

1. **`Generate_Salt()` function** - Creates a fresh random 16-byte salt when none is provided
2. **`Normalize_Salt()` function** - Validates and converts salt input:
   - `None` → generates random salt
   - `str` → encodes to bytes
   - `bytes` → validates it's exactly 16 bytes
   - Anything else → raises TypeError

3. **Sign() updated** - Now uses `Normalize_Salt()` to handle all salt types consistently with proper validation and debug output

4. **Tests demonstrate:**
   - Auto-generated salt (no input needed)
   - Explicit bytes salt (fixed format)
   - String salt (automatic conversion)
   - Different salts produce different signatures
   - Salt validation catches errors (wrong length fails)

The salt handling is now robust, flexible, and prevents accidental misuse.

In [9]:
def Generate_Salt(debug=False):
    salt = bytes([randint(0, 255) for _ in range(16)])
    if debug:
        print(f"[SIGN] Generated random salt: {salt.hex()}")
    return salt

def Normalize_Salt(salt, debug=False):
    if salt is None:
        return Generate_Salt(debug=debug)
    if isinstance(salt, str):
        normalized = salt.encode()
        if debug:
            print(f"[SIGN] Converted string salt to bytes: {normalized.hex()}")
        return normalized
    if isinstance(salt, bytes):
        if len(salt) != 16:
            raise ValueError(f"Salt must be 16 bytes, got {len(salt)}")
        if debug:
            print(f"[SIGN] Using provided salt: {salt.hex()}")
        return salt
    raise TypeError("Salt must be None, str, or bytes")

In [10]:
def Sign(P, O, O_bar, message, salt=None, debug=False):
    F_q = O.base_ring()
    n = O.nrows() + O.ncols()
    m = O.ncols()

    s: Any = None
    
    if debug:
        print(f"\n[SIGN] Starting sign process")
        print(f"[SIGN] n={n}, m={m}, q={F_q.order()}")
        print(f"[SIGN] O shape: {O.nrows()}x{O.ncols()}")
        print(f"[SIGN] O_bar shape: {O_bar.nrows()}x{O_bar.ncols()}")
    
    salt = Normalize_Salt(salt, debug=debug)
    
    t = Hash_Message(message, salt, m, F_q.order(), debug=debug)
    
    if debug:
        print(f"[SIGN] Target hash t: {t[:5]}... (first 5 elements)")
    
    S = [(PM[0]+PM[0].transpose())*O+PM[1] for PM in P]
    
    if debug:
        print(f"[SIGN] Computed {len(S)} S matrices")
    
    signed = False
    attempts = 1
    max_attempts = 256
    
    while not signed:
        if debug and attempts <= 2:
            print(f"\n[SIGN] Attempt {attempts}:")
        
        v = random_matrix(F_q, 1, n-m)
        L = zero_matrix(F_q, m, m)
        y_list = []
        
        for i in range(m):
            L[i] = v * S[i]
            temp = v*P[i][0]*v.transpose()
            y_list.append(temp[0,0])
        
        if debug and attempts <= 2:
            print(f"[SIGN]   v (first 5): {[v[0,i] for i in range(min(5, n-m))]}")
            print(f"[SIGN]   y_list (first 5): {y_list[:5]}")
            print(f"[SIGN]   L rank: {L.rank()}, L determinant: {L.determinant()}")
        
        t_y = vector(t) - vector(y_list)
        
        if debug and attempts <= 2:
            print(f"[SIGN]   t-y: {t_y[:5]}... (first 5)")
        
        try:
            x = L.solve_right(t_y)
            s = vector([v[0, i] for i in range(n-m)] + [0 for i in range(m)]) + O_bar*x
            
            if debug:
                print(f"[SIGN]   Solved! x (first 5): {x[:5]}")
                print(f"[SIGN]   Signature s (first 5): {s[:5]}")
            
            signed = True
        except ValueError:
            if debug and attempts <= 2:
                print(f"[SIGN]   L not invertible, retrying...")
            attempts += 1
            if attempts > max_attempts:
                raise Exception("Signing failed: couldn't find invertible system after many attempts")
    
    if debug:
        print(f"[SIGN] Signature generated successfully in {attempts} attempt(s)\n")
    
    return (s, salt, attempts)


Let's test all the changes

In [11]:
n = 112
m = 44
q = 256

F_q, P, Public_key, O, O_bar = Get_UOV_Keys(n, m, q)

print("="*70)
print("TEST 1: Auto-generated salt")
print("="*70)

message = "Hello, UOV!"
signature, salt, attempts = Sign(P, O, O_bar, message, debug=True)

print(f"Signature generated in {attempts} attempt(s)")
print(f"Auto-generated salt: {salt.hex()}")

is_valid = Verify(Public_key, message, (signature, salt), debug=False)
print(f"Signature valid: {is_valid}\n")

print("="*70)
print("TEST 2: Explicit bytes salt")
print("="*70)

custom_salt = bytes.fromhex("0123456789abcdef0123456789abcdef")
signature2, salt2, attempts2 = Sign(P, O, O_bar, message, salt=custom_salt, debug=True)

print(f"Signature generated in {attempts2} attempt(s)")
print(f"Explicit salt used: {salt2.hex()}")

is_valid2 = Verify(Public_key, message, (signature2, salt2), debug=False)
print(f"Signature valid: {is_valid2}\n")

print("="*70)
print("TEST 3: String salt (auto-converted to bytes)")
print("="*70)

string_salt = "my_custom_salt_1"
signature3, salt3, attempts3 = Sign(P, O, O_bar, message, salt=string_salt, debug=True)

print(f"Signature generated in {attempts3} attempt(s)")
print(f"String salt converted to: {salt3.hex()}")

is_valid3 = Verify(Public_key, message, (signature3, salt3), debug=False)
print(f"Signature valid: {is_valid3}\n")

print("="*70)
print("TEST 4: Different salts produce different signatures")
print("="*70)

salt_a = bytes.fromhex("aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa")
salt_b = bytes.fromhex("bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb")

sig_a, _, _ = Sign(P, O, O_bar, message, salt=salt_a, debug=False)
sig_b, _, _ = Sign(P, O, O_bar, message, salt=salt_b, debug=False)

print(f"Signature with salt_a (first 5 elements): {sig_a[:5]}")
print(f"Signature with salt_b (first 5 elements): {sig_b[:5]}")
print(f"Signatures are different: {sig_a != sig_b}\n")

print("="*70)
print("TEST 5: Salt validation - wrong length fails")
print("="*70)

try:
    bad_salt = bytes.fromhex("0123456789abcdef")
    Sign(P, O, O_bar, message, salt=bad_salt, debug=False)
    print("ERROR: Should have raised ValueError!")
except ValueError as e:
    print(f"Correctly caught error: {e}\n")

TEST 1: Auto-generated salt

[SIGN] Starting sign process
[SIGN] n=112, m=44, q=256
[SIGN] O shape: 68x44
[SIGN] O_bar shape: 112x44
[SIGN] Generated random salt: 3229b7a1923043c96c19327a4e74f301
[HASH] Message: Hello, UOV!
[HASH] Salt: 3229b7a1923043c96c19327a4e74f301
[HASH] Combined: 48656c6c6f2c20554f56213229b7a1923043c96c19327a4e74f301
[HASH] Hash bytes: d07ea51618c47fe51187c1ca20141baa7784a7d0c2686f9aaae15401ce26ad14ebe9161ff8638cd6b597cab42985591749dadb64225a1ce48ae389a85312e429b4c5aabaecc7d804c8052b1f22ba05e34065daebea76c56e
[HASH] Element 0: byte_val=53374, field_elem=0
[HASH] Element 1: byte_val=42262, field_elem=0
[HASH] Element 2: byte_val=6340, field_elem=0
[HASH] Element 3: byte_val=32741, field_elem=1
[HASH] Element 4: byte_val=4487, field_elem=1
[HASH] Final hash vector: (0, 0, 0, 1, 1)... (showing first 5)
[SIGN] Target hash t: (0, 0, 0, 1, 1)... (first 5 elements)
[SIGN] Computed 44 S matrices

[SIGN] Attempt 1:
[SIGN]   v (first 5): [z8^7 + z8^5 + z8, z8^7 + z8^6 + z8

Output is exactly what we expected. 

## 3. performance optimization with S matrix caching

**Get_UOV_Keys():**
- Now computes S matrices during key generation and returns them as `S_precomputed`
- S_i = (P1_i + P1_i^T) * O + P2_i is computed once, not repeatedly

**Sign():**
- Added `S_precomputed` parameter (optional)
- If provided, uses precomputed S matrices
- If None, computes them on-the-fly (backward compatible)
- Debug output shows which path is being used

**Tests:**
- TEST 1: Sign using precomputed S matrices
- TEST 2: Sign computing S matrices on-the-fly
- TEST 3: Performance comparison across 5 signatures, shows speedup factor

The S matrices are expensive to compute (matrix multiplications), so precomputing them once during key generation and reusing them for all signatures is much faster.

In [ ]:
def Get_UOV_Keys(n, m, q):
    o = m
    F_q = GF(q)
    O = random_matrix(F_q, n-o, o)
    O_bar = block_matrix([[O], [identity_matrix(F_q, o)]])
    
    P = []
    Public_key = []
    S_precomputed = []
    
    for i in range(0, m):
        P1 = Upper(random_matrix(F_q, n-o, n-o))
        P2 = random_matrix(F_q, n-o, o)
        P3 = Upper(-O.transpose()*P1*O - O.transpose()*P2)
        
        P.append([P1, P2, P3])
        
        matriz = block_matrix([[P1, P2], [zero_matrix(F_q, o, n-o), P3]])
        Public_key.append(matriz)
        
        S_i = (P1 + P1.transpose()) * O + P2
        S_precomputed.append(S_i)
    
    return F_q, P, Public_key, O, O_bar, S_precomputed


def Sign(P, O, O_bar, message, S_precomputed=None, salt=None, debug=False):
    F_q = O.base_ring()
    n = O.nrows() + O.ncols()
    m = O.ncols()

    s: Any = None
    
    if debug:
        print(f"\n[SIGN] Starting sign process")
        print(f"[SIGN] n={n}, m={m}, q={F_q.order()}")
    
    salt = Normalize_Salt(salt, debug=debug)
    
    t = Hash_Message(message, salt, m, F_q.order(), debug=debug)
    
    if S_precomputed is None:
        S = [(PM[0]+PM[0].transpose())*O+PM[1] for PM in P]
        if debug:
            print(f"[SIGN] Computed {len(S)} S matrices on-the-fly")
    else:
        S = S_precomputed
        if debug:
            print(f"[SIGN] Using {len(S)} precomputed S matrices")
    
    signed = False
    attempts = 1
    max_attempts = 256
    
    while not signed:
        v = random_matrix(F_q, 1, n-m)
        L = zero_matrix(F_q, m, m)
        y_list = []
        
        for i in range(m):
            L[i] = v * S[i]
            temp = v*P[i][0]*v.transpose()
            y_list.append(temp[0,0])
        
        t_y = vector(t) - vector(y_list)
        
        try:
            x = L.solve_right(t_y)
            s = vector([v[0, i] for i in range(n-m)] + [0 for i in range(m)]) + O_bar*x
            signed = True
        except ValueError:
            attempts += 1
            if attempts > max_attempts:
                raise Exception("Signing failed: couldn't find invertible system after many attempts")
    
    return (s, salt, attempts)

In [23]:
import time

n = 112
m = 44
q = 256

F_q, P, Public_key, O, O_bar, S_precomputed = Get_UOV_Keys(n, m, q)

print("="*70)
print("TEST 1: Performance comparison - precomputed vs on-the-fly S matrices")
print("="*70)

num_signatures = 5

print(f"\nWith precomputed S matrices:")
start = time.time()
for i in range(num_signatures):
    sig, _, _ = Sign(P, O, O_bar, f"Message {i}", S_precomputed=S_precomputed, debug=False)
time_precomputed = time.time() - start
print(f"  Time for {num_signatures} signatures: {time_precomputed:.3f}s")
print(f"  Average per signature: {time_precomputed/num_signatures:.3f}s")

print(f"\nWithout precomputed S matrices (on-the-fly):")
start = time.time()
for i in range(num_signatures):
    sig, _, _ = Sign(P, O, O_bar, f"Message {i}", S_precomputed=None, debug=False)
time_computed = time.time() - start
print(f"  Time for {num_signatures} signatures: {time_computed:.3f}s")
print(f"  Average per signature: {time_computed/num_signatures:.3f}s")

speedup = time_computed / time_precomputed
print(f"\nSpeedup with precomputed S: {speedup:.2f}x faster")

TEST 1: Performance comparison - precomputed vs on-the-fly S matrices

With precomputed S matrices:
  Time for 5 signatures: 0.037s
  Average per signature: 0.007s

Without precomputed S matrices (on-the-fly):
  Time for 5 signatures: 0.194s
  Average per signature: 0.039s

Speedup with precomputed S: 5.20x faster


## 4. Better Sign/Verify interface

**What changed:**

1. **`UOVSignatureScheme` class** - Encapsulates the entire scheme:
   - `__init__()` - Generates keys on initialization
   - `sign(message, salt=None)` - Simple signing interface
   - `verify(message, signature)` - Simple verification interface
   - `get_public_key()` - Access public key
   - `get_secret_key()` - Access secret key

**Tests show:**
- TEST 1: Clean sign/verify with natural interface
- TEST 2: Multiple messages work seamlessly
- TEST 3: Tampering detection (wrong message, modified signature both fail)
- TEST 4: Key access methods

Much cleaner than passing individual components around.

In [14]:
class UOVSignatureScheme:
    def __init__(self, n, m, q):
        self.n = n
        self.m = m
        self.q = q
        self.F_q, self.P, self.Public_key, self.O, self.O_bar, self.S_precomputed = Get_UOV_Keys(n, m, q)
    
    def sign(self, message, salt=None):
        s, salt, attempts = Sign(self.P, self.O, self.O_bar, message, S_precomputed=self.S_precomputed, salt=salt, debug=False)
        return (s, salt)
    
    def verify(self, message, signature):
        return Verify(self.Public_key, message, signature, debug=False)
    
    def get_public_key(self):
        return self.Public_key
    
    def get_secret_key(self):
        return (self.P, self.O, self.O_bar, self.S_precomputed)

n = 112
m = 44
q = 256

uov = UOVSignatureScheme(n, m, q)

print("="*70)
print("TEST 1: Simple sign and verify with new interface")
print("="*70)

message = "Hello, UOV!"
signature = uov.sign(message)
is_valid = uov.verify(message, signature)

print(f"Message: {message}")
print(f"Signature: ({signature[0][:3]}..., {signature[1].hex()})")
print(f"Valid: {is_valid}\n")

print("="*70)
print("TEST 2: Multiple messages")
print("="*70)

messages = ["Message 1", "Message 2", "Message 3"]
signatures = []

for msg in messages:
    sig = uov.sign(msg)
    signatures.append(sig)
    is_valid = uov.verify(msg, sig)
    print(f"Message: '{msg}' -> Valid: {is_valid}")

print()

print("="*70)
print("TEST 3: Signature tampering detection")
print("="*70)

message = "Important message"
sig, salt = uov.sign(message)

is_valid_original = uov.verify(message, (sig, salt))
print(f"Original signature valid: {is_valid_original}")

is_valid_wrong_msg = uov.verify("Modified message", (sig, salt))
print(f"Wrong message with same signature: {is_valid_wrong_msg}")

F_q = uov.F_q
tampered_sig = (sig + vector([F_q(0)] * (len(sig) - 1) + [F_q(1)]), salt)
is_valid_tampered = uov.verify(message, tampered_sig)
print(f"Tampered signature: {is_valid_tampered}\n")

print("="*70)
print("TEST 4: Key access")
print("="*70)

public_key = uov.get_public_key()
secret_key = uov.get_secret_key()

print(f"Public key polynomials: {len(public_key)}")
print(f"Secret key components: {len(secret_key)}")
print(f"Secret key contains P, O, O_bar, S_precomputed: {len(secret_key) == 4}")

TEST 1: Simple sign and verify with new interface
Message: Hello, UOV!
Signature: ((z8^6 + z8^5 + z8^4 + z8^2 + z8 + 1, z8^6 + z8^4 + z8^3 + z8^2 + z8 + 1, z8^7 + z8^6 + z8^2 + z8 + 1)..., 51d4a431e67111b215f25e76aeb13249)
Valid: True

TEST 2: Multiple messages
Message: 'Message 1' -> Valid: True
Message: 'Message 2' -> Valid: True
Message: 'Message 3' -> Valid: True

TEST 3: Signature tampering detection
Original signature valid: True
Wrong message with same signature: False
Tampered signature: False

TEST 4: Key access
Public key polynomials: 44
Secret key components: 4
Secret key contains P, O, O_bar, S_precomputed: True


## 5. Benchmark

In [15]:
def Benchmark(n, m, q, num_signatures=10):
    print(f"\nBenchmarking UOV with parameters: n={n}, m={m}, q={q}")
    print("="*70)
    
    uov = UOVSignatureScheme(n, m, q)
    
    print(f"\nKey Generation Time:")
    print(f"  (included in UOVSignatureScheme initialization)")
    print(f"  Parameters: n={n}, m={m}, q={q}")
    print(f"  Public key size: {len(uov.Public_key)} polynomials")
    
    print(f"\nSigning Performance ({num_signatures} signatures):")
    messages = [f"Test message {i}" for i in range(num_signatures)]
    
    start = time.time()
    signatures = []
    for msg in messages:
        sig = uov.sign(msg)
        signatures.append((msg, sig))
    sign_time = time.time() - start
    
    avg_sign_time = sign_time / num_signatures
    print(f"  Total time: {sign_time:.4f}s")
    print(f"  Average per signature: {avg_sign_time:.4f}s")
    print(f"  Signatures per second: {1/avg_sign_time:.2f}")
    
    print(f"\nVerification Performance ({num_signatures} signatures):")
    start = time.time()
    valid_count = 0
    for msg, sig in signatures:
        if uov.verify(msg, sig):
            valid_count += 1
    verify_time = time.time() - start
    
    avg_verify_time = verify_time / num_signatures
    print(f"  Total time: {verify_time:.4f}s")
    print(f"  Average per signature: {avg_verify_time:.4f}s")
    print(f"  Verifications per second: {1/avg_verify_time:.2f}")
    print(f"  Valid signatures: {valid_count}/{num_signatures}")
    
    print(f"\nSummary:")
    print(f"  Sign/Verify ratio: {avg_sign_time/avg_verify_time:.2f}x")
    print(f"  Total operations: {num_signatures * 2} (sign + verify)")
    print(f"  Total time: {sign_time + verify_time:.4f}s")
    
    return {
        'n': n,
        'm': m,
        'q': q,
        'sign_time': avg_sign_time,
        'verify_time': avg_verify_time,
        'valid_signatures': valid_count
    }

def Benchmark_Multiple_Parameters():
    print("\n" + "="*70)
    print("COMPREHENSIVE BENCHMARK - Multiple Parameter Sets")
    print("="*70)
    
    parameter_sets = [
        (112, 44, 256),
        (160, 64, 16),
        (184, 72, 256),
    ]
    
    results = []
    
    for n, m, q in parameter_sets:
        result = Benchmark(n, m, q, num_signatures=5)
        results.append(result)
    
    print("\n" + "="*70)
    print("COMPARISON TABLE")
    print("="*70)
    print(f"{'(n, m, q)':<15} {'Avg Sign (ms)':<15} {'Avg Verify (ms)':<15} {'Ratio':<10}")
    print("-"*70)
    
    for result in results:
        n, m, q = result['n'], result['m'], result['q']
        sign_ms = result['sign_time'] * 1000
        verify_ms = result['verify_time'] * 1000
        ratio = result['sign_time'] / result['verify_time']
        print(f"({n}, {m}, {q}){' ':<5} {sign_ms:<15.2f} {verify_ms:<15.2f} {ratio:<10.2f}x")
    
    print("="*70)

In [16]:
n = 112
m = 44
q = 256

uov = UOVSignatureScheme(n, m, q)

print("="*70)
print("TEST 1: Single parameter set benchmark")
print("="*70)

Benchmark(n, m, q, num_signatures=10)

print("\n" + "="*70)
print("TEST 2: Multiple parameter sets comparison")
print("="*70)

Benchmark_Multiple_Parameters()

TEST 1: Single parameter set benchmark

Benchmarking UOV with parameters: n=112, m=44, q=256

Key Generation Time:
  (included in UOVSignatureScheme initialization)
  Parameters: n=112, m=44, q=256
  Public key size: 44 polynomials

Signing Performance (10 signatures):
  Total time: 0.0822s
  Average per signature: 0.0082s
  Signatures per second: 121.67

Verification Performance (10 signatures):
  Total time: 0.2187s
  Average per signature: 0.0219s
  Verifications per second: 45.73
  Valid signatures: 10/10

Summary:
  Sign/Verify ratio: 0.38x
  Total operations: 20 (sign + verify)
  Total time: 0.3009s

TEST 2: Multiple parameter sets comparison

COMPREHENSIVE BENCHMARK - Multiple Parameter Sets

Benchmarking UOV with parameters: n=112, m=44, q=256

Key Generation Time:
  (included in UOVSignatureScheme initialization)
  Parameters: n=112, m=44, q=256
  Public key size: 44 polynomials

Signing Performance (5 signatures):
  Total time: 0.0372s
  Average per signature: 0.0074s
  Signa